In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/ue21cs342aa2/sample_submission.csv
/kaggle/input/ue21cs342aa2/train.csv
/kaggle/input/ue21cs342aa2/test.csv


In [3]:
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import VotingClassifier

# Read the train and test data
train_data = pd.read_csv("/kaggle/input/ue21cs342aa2/train.csv")
test_data = pd.read_csv("/kaggle/input/ue21cs342aa2/test.csv")

In [4]:
# Convert the 'Date' column to a datetime object
train_data['Date'] = pd.to_datetime(train_data['Date'])
test_data['Date'] = pd.to_datetime(test_data['Date'])

# Sort the data by date
train_data.sort_values(by='Date', inplace=True)
test_data.sort_values(by='Date', inplace=True)

In [5]:
# Calculate the differenced 'Close' values to make the data stationary
train_data['Close_Diff'] = train_data['Close'].diff()
train_data = train_data.dropna()

In [6]:
train_data

,id,Date,Open,Close,Volume,Strategy,Close_Diff
1,1,2015-06-08,64.116235,74.443331,227974800,Hold,-1.166647
2,2,2015-06-15,47.701942,71.140831,250670900,Buy,-3.302500
3,3,2015-06-22,54.754816,72.881344,223614300,Hold,1.740512
4,4,2015-06-29,48.031899,66.284718,406814900,Buy,-6.596626
5,5,2015-07-06,51.374399,58.494017,590857400,Buy,-7.790701
...,...,...,...,...,...,...,...
295,295,2021-01-25,81.284821,83.373498,117281600,Sell,-0.516252
296,296,2021-02-01,71.970249,85.294903,177655800,Sell,1.921405
297,297,2021-02-08,71.784627,83.266453,146003500,Hold,-2.028450
298,298,2021-02-15,69.372333,81.026827,167762500,Hold,-2.239626


In [7]:
# Train the SARIMAX model for 'Close'
order = (0, 1, 1)  # Example order, you may need to fine-tune
seasonal_order = (1, 0, 1, 12)  # Example seasonal order, you may need to fine-tune

sarimax_model = SARIMAX(train_data['Close_Diff'], order=order, seasonal_order=seasonal_order)
sarimax_result = sarimax_model.fit()

# Forecast 'Close' for the main test_data using the SARIMAX model
predicted_close_diff = sarimax_result.predict(start=len(train_data), end=len(train_data) + len(test_data) - 1, dynamic=False)
predicted_close = train_data['Close'].iloc[-1] + predicted_close_diff.cumsum()


/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: An unsupported index was provided and will be ignored when e.g. forecasting.
  self._init_dates(dates, freq)
 This problem is unconstrained.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            4     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  2.65130D+00    |proj g|=  4.97201D-02

At iterate    5    f=  2.60506D+00    |proj g|=  1.62246D-02

At iterate   10    f=  2.58922D+00    |proj g|=  2.12255D-03

At iterate   15    f=  2.58864D+00    |proj g|=  5.10225D-03

At iterate   20    f=  2.58792D+00    |proj g|=  7.53113D-04

At iterate   25    f=  2.58787D+00    |proj g|=  1.28848D-04

At iterate   30    f=  2.58786D+00    |proj g|=  1.28622D-04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nac

/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
/opt/conda/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:836: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


In [8]:
predicted_close.describe()
predicted_close = predicted_close.reset_index(drop=True)

In [9]:
predicted_close

0     79.393125
1     78.911474
2     78.246025
3     79.242772
4     78.837888
        ...    
95    80.176853
96    80.231646
97    80.212943
98    80.183008
99    80.254644
Name: predicted_mean, Length: 100, dtype: float64

In [11]:
!pip install pmdarima

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 33.3 MB/s eta 0:00:00a 0:00:01


In [12]:
import pmdarima as pm

# Assuming you have your time series data in 'train_data'
model = pm.auto_arima(train_data['Close'], seasonal=True, m=12, stepwise=True, trace=True)

# Print the best model's order and seasonal_order
print("Best order:", model.order)
print("Best seasonal_order:", model.seasonal_order)


Performing stepwise search to minimize aic
 ARIMA(2,1,2)(1,0,1)[12] intercept   : AIC=1550.387, Time=4.18 sec
 ARIMA(0,1,0)(0,0,0)[12] intercept   : AIC=1555.125, Time=0.06 sec
 ARIMA(1,1,0)(1,0,0)[12] intercept   : AIC=1552.284, Time=0.36 sec
 ARIMA(0,1,1)(0,0,1)[12] intercept   : AIC=1551.713, Time=0.41 sec
 ARIMA(0,1,0)(0,0,0)[12]             : AIC=1553.130, Time=0.04 sec
 ARIMA(2,1,2)(0,0,1)[12] intercept   : AIC=1555.837, Time=2.65 sec
 ARIMA(2,1,2)(1,0,0)[12] intercept   : AIC=1556.360, Time=2.99 sec
 ARIMA(2,1,2)(2,0,1)[12] intercept   : AIC=1551.925, Time=8.08 sec
 ARIMA(2,1,2)(1,0,2)[12] intercept   : AIC=1551.912, Time=7.05 sec
 ARIMA(2,1,2)(0,0,0)[12] intercept   : AIC=inf, Time=0.64 sec
 ARIMA(2,1,2)(0,0,2)[12] intercept   : AIC=1553.899, Time=4.71 sec
 ARIMA(2,1,2)(2,0,0)[12] intercept   : AIC=1555.082, Time=6.05 sec
 ARIMA(2,1,2)(2,0,2)[12] intercept   : AIC=inf, Time=8.27 sec
 ARIMA(1,1,2)(1,0,1)[12] intercept   : AIC=inf, Time=2.90 sec
 ARIMA(2,1,1)(1,0,1)[12] intercept

In [13]:
predicted_close.describe()

count    100.000000
mean      79.701142
std        0.421399
min       78.246025
25%       79.483396
50%       79.786846
75%       80.056214
max       80.421650
Name: predicted_mean, dtype: float64

In [14]:
test_data['Close']=predicted_close
test_data

,id,Date,Open,Volume,Close
0,0,2021-03-01,72.708331,205897600,79.393125
1,1,2021-03-08,73.310886,214095600,78.911474
2,2,2021-03-15,70.610119,151601700,78.246025
3,3,2021-03-22,70.256017,193982500,79.242772
4,4,2021-03-29,68.258324,220238400,78.837888
...,...,...,...,...,...
95,95,2022-12-26,57.048622,224994700,80.176853
96,96,2023-01-02,60.569661,176020500,80.231646
97,97,2023-01-09,61.446777,181478600,80.212943
98,98,2023-01-16,55.447711,162471700,80.183008


In [15]:
test_data['Close'] = predicted_close

In [16]:
test_data

,id,Date,Open,Volume,Close
0,0,2021-03-01,72.708331,205897600,79.393125
1,1,2021-03-08,73.310886,214095600,78.911474
2,2,2021-03-15,70.610119,151601700,78.246025
3,3,2021-03-22,70.256017,193982500,79.242772
4,4,2021-03-29,68.258324,220238400,78.837888
...,...,...,...,...,...
95,95,2022-12-26,57.048622,224994700,80.176853
96,96,2023-01-02,60.569661,176020500,80.231646
97,97,2023-01-09,61.446777,181478600,80.212943
98,98,2023-01-16,55.447711,162471700,80.183008


In [17]:
test_data.describe()

,id,Date,Open,Volume,Close
count,100.000000,100,100.000000,1.000000e+02,100.000000
mean,49.500000,2022-02-10 12:00:00,70.265016,1.975365e+08,79.701142
min,0.000000,2021-03-01 00:00:00,51.825792,6.182620e+07,78.246025
25%,24.750000,2021-08-21 06:00:00,63.318829,1.584331e+08,79.483396
50%,49.500000,2022-02-10 12:00:00,71.470649,1.928077e+08,79.786846
75%,74.250000,2022-08-02 18:00:00,77.451497,2.263687e+08,80.056214
max,99.000000,2023-01-23 00:00:00,89.752238,4.508312e+08,80.421650
std,29.011492,NaN,9.010469,5.856774e+07,0.421399


In [18]:
# Train a Random Forest model for 'Strategy'
rf_model = RandomForestClassifier(n_estimators=100, random_state=0)
rf_model.fit(train_data[['Open', 'Close', 'Volume']], train_data['Strategy'])
predicted_strategy_rf = rf_model.predict(test_data[['Open', 'Close', 'Volume']])

# Train a K-Nearest Neighbors model for 'Strategy'
knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(train_data[['Open', 'Close', 'Volume']], train_data['Strategy'])
predicted_strategy_knn = knn_model.predict(test_data[['Open', 'Close', 'Volume']])


In [19]:
voting_model = VotingClassifier(estimators=[('rf', rf_model), ('knn', knn_model)], voting='soft')
voting_model.fit(train_data[['Open', 'Close', 'Volume']], train_data['Strategy'])
predicted_strategy = voting_model.predict(test_data[['Open', 'Close', 'Volume']])

In [20]:
submission_new = test_data[['id','Date']].copy()
submission_new['Close'] = predicted_close
submission_new['Strategy'] = predicted_strategy

In [21]:
predicted_strategy

array(['Hold', 'Buy', 'Buy', 'Buy', 'Hold', 'Buy', 'Buy', 'Buy', 'Buy',
       'Hold', 'Buy', 'Hold', 'Hold', 'Hold', 'Hold', 'Buy', 'Hold',
       'Sell', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold',
       'Hold', 'Hold', 'Hold', 'Buy', 'Hold', 'Hold', 'Hold', 'Hold',
       'Hold', 'Hold', 'Hold', 'Hold', 'Buy', 'Hold', 'Hold', 'Hold',
       'Buy', 'Buy', 'Hold', 'Buy', 'Hold', 'Hold', 'Sell', 'Buy', 'Hold',
       'Hold', 'Hold', 'Hold', 'Buy', 'Buy', 'Hold', 'Hold', 'Hold',
       'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Buy', 'Hold', 'Buy',
       'Sell', 'Hold', 'Sell', 'Buy', 'Hold', 'Hold', 'Hold', 'Sell',
       'Sell', 'Hold', 'Hold', 'Buy', 'Hold', 'Hold', 'Sell', 'Hold',
       'Hold', 'Buy', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold', 'Hold',
       'Buy', 'Hold', 'Sell', 'Hold', 'Hold', 'Hold', 'Sell', 'Hold',
       'Hold', 'Hold'], dtype=object)

In [22]:
submission_new.to_csv('submission.csv', index=False)